### Import libraries

In [22]:
from IPython.core.display import display
from src.utils.preprocessing import classic_preprocessing
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd
import numpy as np

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [24]:
DATA_PATH = '../../../data'
X_coarse, y_coarse = import_data(DATA_PATH, segmentation_type='coarse',
                                 drop_user_features=True,
                                 drop_expert=True)

In [25]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='coarse',
                       drop_user_features=True,
                       drop_expert=False)

In [26]:
display(X_coarse.head())
display(y_coarse.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               8.0          8.0   
                                     1               2.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0               4.0          4.0   
                                     1               3.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0               6.0          6.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          6.0   
                                     1                2.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                4.0          4.0   
                                     1                5.0          6.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          7.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          9.0   
                                     1                2.0          2.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                6.0          6.0   
                                     1                4.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          7.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          7.0   
                                     1                3.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                5.0          5.0   
                                     1                6.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          6.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          8.0  ...   
                                     1                3.0          3.0  ...   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                5.0          5.0  ...   
                                     1                5.0          4.0  ...   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                8.0          6.0  ...   

                                              MFCC_std11  MFCC_std12  \
subject                              file_id                           
008ba489-31ad-44d8-856b-fcf72369dc46 0         14.163379   10.952018   
                                     1         11.684959    9.106066   
008c1c9e-aeef-40c5-846c-24f1b964f884 0         13.539921    8.786539   
                                     1         12.846291    7.035100   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0         14.501091    9.551842   

                                              Crest_Factor  Cough_Length  \
subject                              file_id                               
008ba489-31ad-44d8-856b-fcf72369dc46 0            8.365480      1.036563   
                                     1            9.348207      0.378625   
008c1c9e-aeef-40c5-846c-24f1b964f884 0            5.222938      0.765625   
                                     1            4.402501      0.794125   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0            4.234178      0.860250   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.314239     0.027049   
                                     1           0.093550     0.107780   
008c1c9e-aeef-40c5-846c-24f1b964f884 0           0.160032     0.034572   
               

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
008c1c9e-aeef-40c5-846c-24f1b964f884 0          1.0
                                     1          1.0
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0          1.0

### Preprocessing

In [27]:
X_coarse = classic_preprocessing(X_coarse, stop=None, dummy=False)

In [28]:
# Save the expert feature for split
expert = X_e['Expert'].copy()
X_e = classic_preprocessing(X_e, stop=None, dummy=False)
X_e['Expert'] = expert.values

### Split expert models

In [29]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

#### 1. Logistic regression

In [30]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  accuracy_score
max_iter oversampling                                         
10000    True          0.441111       0.597721        0.601744
         False         0.173302       0.530912        0.728295
100000   True          0.441111       0.597721        0.601744
         False         0.173302       0.530912        0.728295

In [31]:
# Best model parameters

best_log = log_results.iloc[[log_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_log)

,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.441111,0.597721,0.601744


In [32]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=5,
                                         params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=5,
                                         params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=5,
                                         params=LOGISTIC_PARAMS)

In [33]:
# Best model parameters

best_log_1 = log_results_1.iloc[[
    log_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_log_2 = log_results_2.iloc[[
    log_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_log_3 = log_results_3.iloc[[
    log_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_log_1, best_log_2, best_log_3)

,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.383782,0.664399,0.73355


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.319093,0.564735,0.600532


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,False,0.630736,0.645493,0.644571


#### 2. Linear Discriminant Analysis

In [34]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.442533,0.598766,0.601550
False,0.174198,0.531034,0.728101


In [35]:
# Best model parameters

best_lda = lda_results.iloc[[lda_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.442533,0.598766,0.60155


In [36]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=5,
                                         params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=5,
                                         params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=5,
                                         params=LDA_PARAMS)

In [37]:
best_lda_1 = lda_results_1.iloc[[
    lda_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_lda_2 = lda_results_2.iloc[[
    lda_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_lda_3 = lda_results_3.iloc[[
    lda_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.399307,0.685658,0.719218


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.31335,0.557742,0.587766


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
False,0.621571,0.638918,0.638286


#### 3. K-nearest Neighbors

In [38]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  accuracy_score
n_neighbors oversampling                                         
1           True          0.441204       0.596747        0.593411
            False         0.400194       0.590123        0.675194
2           True          0.418408       0.591236        0.638566
            False         0.222911       0.542174        0.726163
3           True          0.443599       0.585158        0.530039
            False         0.348454       0.572943        0.698643
4           True          0.434848       0.587257        0.574612
            False         0.206883       0.537963        0.725581
5           True          0.437869       0.571699        0.492636
            False         0.282748       0.550141        0.704457
6           True          0.430461       0.573060        0.529070
            False         0.190812       0.533693        0.725000
7           True          0.435825       0.565174        0.472481
            False         0.235355       0.535360        0.704651
8           True          0.428693       0.565163        0.501744
            False         0.149075       0.521909        0.722093
9           True          0.437859       0.564727        0.460853
            False         0.209546       0.529969        0.708721
10          True          0.435427       0.568772        0.489729
            False         0.147613       0.522418        0.724031
11          True          0.435436       0.559032        0.446512
            False         0.190946       0.527412        0.713953
12          True          0.434967       0.564994        0.475194
            False         0.132800       0.519324        0.724225
13          True          0.435533       0.557429        0.438760
            False         0.174064       0.525617        0.718411
14          True          0.430587       0.556815        0.457558
            False         0.120255       0.516868        0.724612
15          True          0.433677       0.553826        0.430814
            False         0.150617       0.520941        0.720155

In [39]:
# Best model parameters

best_knn = knn_results.iloc[[knn_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.441204,0.596747,0.593411


In [40]:
knn_results_1 = hyperparameter_tuning_cv(
    model='knn', data=X_e_1, labels=y_e_1, cv_k=5, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(
    model='knn', data=X_e_2, labels=y_e_2, cv_k=5, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(
    model='knn', data=X_e_3, labels=y_e_3, cv_k=5, params=KNN_PARAMS)

In [41]:
best_knn_1 = knn_results_1.iloc[[
    knn_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_knn_2 = knn_results_2.iloc[[
    knn_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_knn_3 = knn_results_3.iloc[[
    knn_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
2,True,0.369292,0.653015,0.728339


,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
2,True,0.319354,0.573739,0.670213


,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
3,False,0.626356,0.651425,0.651429


#### 4. Support Vector Classifier

In [42]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                       params=SVC_PARAMS)

display(svc_results)

f1_score  roc_auc_score  accuracy_score
kernel gamma oversampling                                         
linear 0.10  True          0.444824       0.598347        0.590698
             False         0.000000       0.500000        0.733140
rbf    0.10  True          0.148567       0.530659        0.737791
             False         0.025850       0.505557        0.735078
linear 0.01  True          0.444824       0.598347        0.590698
             False         0.000000       0.500000        0.733140
rbf    0.01  True          0.433707       0.598158        0.626550
             False         0.004429       0.500978        0.733527

In [43]:
# Best model parameters

best_svc = svc_results.iloc[[svc_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_svc)

,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
linear,0.1,True,0.444824,0.598347,0.590698


In [44]:
svc_results_1 = hyperparameter_tuning_cv(
    model='svc', data=X_e_1, labels=y_e_1, cv_k=5, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(
    model='svc', data=X_e_2, labels=y_e_2, cv_k=5, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(
    model='svc', data=X_e_3, labels=y_e_3, cv_k=5, params=SVC_PARAMS)

In [45]:
best_svc_1 = svc_results_1.iloc[[
    svc_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_svc_2 = svc_results_2.iloc[[
    svc_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_svc_3 = svc_results_3.iloc[[
    svc_results_3.reset_index()['roc_auc_score'].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
linear,0.1,True,0.385733,0.667996,0.726384


,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,True,0.305217,0.562734,0.665426


,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,False,0.635175,0.657252,0.656571


#### 5. Naive Bayes Classifier

In [46]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                      params=NAIVE_BAYES_PARAMS)

display(nb_results)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.015508,0.499562,0.728682
False,0.019639,0.499352,0.727326


In [47]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_nb)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.015508,0.499562,0.728682


In [48]:
nb_results_1 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=5, params=NAIVE_BAYES_PARAMS)

In [49]:
best_nb_1 = nb_results_1.iloc[[
    nb_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_nb_2 = nb_results_2.iloc[[
    nb_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_nb_3 = nb_results_3.iloc[[
    nb_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.317933,0.606387,0.689251


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
False,0.29969,0.531334,0.504787


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
False,0.258991,0.530763,0.544571


#### 6. Decision Tree

In [50]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                      params=DECISION_TREE_PARAMS)

display(dt_results)

f1_score  roc_auc_score  accuracy_score
max_depth oversampling                                         
3         True          0.393149       0.544657        0.522481
          False         0.083805       0.509947        0.724419
5         True          0.404663       0.551233        0.524031
          False         0.235914       0.537033        0.705620
7         True          0.400694       0.554414        0.550194
          False         0.249376       0.533036        0.688566

In [51]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_dt)

,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
7,True,0.400694,0.554414,0.550194


In [52]:
dt_results_1 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=5, params=DECISION_TREE_PARAMS)

In [53]:
best_dt_1 = dt_results_1.iloc[[
    dt_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_dt_2 = dt_results_2.iloc[[
    dt_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_dt_3 = dt_results_3.iloc[[
    dt_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
5,True,0.34062,0.627875,0.710098


,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
3,True,0.295198,0.525743,0.481383


,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
3,True,0.555246,0.602812,0.605714


#### 7. Random Forest

In [54]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_coarse, labels=y_coarse.Label, cv_k=5,
                                      params=RANDOM_FOREST_PARAMS)

display(rf_results)

f1_score  roc_auc_score  accuracy_score
max_depth n_estimators oversampling                                         
3         3            True          0.398811       0.553846        0.552326
                       False         0.038322       0.504437        0.730233
          5            True          0.411632       0.569956        0.577132
                       False         0.015369       0.502995        0.733915
          7            True          0.413424       0.572887        0.583915
                       False         0.005776       0.499997        0.731783
5         3            True          0.389507       0.551505        0.566667
                       False         0.108584       0.512491        0.721318
          5            True          0.403456       0.566802        0.585271
                       False         0.092169       0.514070        0.729457
          7            True          0.401931       0.570920        0.604845
                       False         0.055791       0.509571        0.733140
7         3            True          0.377376       0.549870        0.586822
                       False         0.191108       0.528264        0.714922
          5            True          0.394439       0.568181        0.609690
                       False         0.140153       0.518980        0.720736
          7            True          0.396024       0.568350        0.608915
                       False         0.125577       0.521231        0.730233

In [55]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_rf)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,7,True,0.413424,0.572887,0.583915


In [56]:
rf_results_1 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_1, labels=y_e_1, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_2, labels=y_e_2, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_3, labels=y_e_3, cv_k=5, params=RANDOM_FOREST_PARAMS)

In [57]:
best_rf_1 = rf_results_1.iloc[[
    rf_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_rf_2 = rf_results_2.iloc[[
    rf_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_rf_3 = rf_results_3.iloc[[
    rf_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,5,True,0.377011,0.66056,0.722476


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
7,5,True,0.297491,0.555652,0.660106


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
7,7,True,0.616167,0.630821,0.630857


#### 8. Gradient Boosting

In [58]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_coarse,
                                      labels=y_coarse.Label, cv_k=5,
                                      params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

f1_score  roc_auc_score  accuracy_score
max_depth n_estimators oversampling                                         
3         3            True          0.401801       0.557102        0.554264
                       False         0.000000       0.500000        0.733140
          5            True          0.412778       0.567826        0.565310
                       False         0.000000       0.500000        0.733140
          7            True          0.414286       0.574660        0.587791
                       False         0.000000       0.500000        0.733140
5         3            True          0.404909       0.565183        0.576938
                       False         0.000000       0.500000        0.733140
          5            True          0.407283       0.571379        0.593411
                       False         0.011341       0.502221        0.733721
          7            True          0.402172       0.569235        0.597868
                       False         0.029883       0.506220        0.735078
7         3            True          0.399905       0.567004        0.595349
                       False         0.001550       0.500262        0.733140
          5            True          0.407131       0.576610        0.611628
                       False         0.041845       0.506507        0.732171
          7            True          0.408496       0.580569        0.624225
                       False         0.086817       0.516214        0.734690

In [59]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_gb)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
7,7,True,0.408496,0.580569,0.624225


In [60]:
gb_results_1 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=5,
    params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=5,
    params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=5,
    params=GRADIENT_BOOSTING_PARAMS)

In [61]:
best_gb_1 = gb_results_1.iloc[[
    gb_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_gb_2 = gb_results_2.iloc[[
    gb_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_gb_3 = gb_results_3.iloc[[
    gb_results_3.reset_index()['roc_auc_score'].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,5,True,0.36284,0.64941,0.712052


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,7,True,0.310685,0.558467,0.601596


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,7,True,0.595377,0.625716,0.626857


### Results

In [62]:
# display('logistic', best_log)
# display('lda', best_lda)
# display('knn', best_knn)
# display('svc', best_svc)
# display('naive_bayes', best_nb)
# display('decision_tree', best_dt)
# display('random_forest', best_rf)
# display('gradient_boosting', best_gb)

In [63]:
weighted_average = (np.array([len(X_e_1), len(X_e_2), len(X_e_3)]) / len(X_e))

e_log = np.sum(pd.concat([best_log_1, best_log_2, best_log_3]
                         ).roc_auc_score.values * weighted_average)
e_lda = np.sum(pd.concat([best_lda_1, best_lda_2, best_lda_3]
                         ).roc_auc_score.values * weighted_average)
e_knn = np.sum(pd.concat([best_knn_1, best_knn_2, best_knn_3]
                         ).roc_auc_score.values * weighted_average)
e_svc = np.sum(pd.concat([best_svc_1, best_svc_2, best_svc_3]
                         ).roc_auc_score.values * weighted_average)
e_nb = np.sum(pd.concat([best_nb_1, best_nb_2, best_nb_3]
                        ).roc_auc_score.values * weighted_average)
e_dt = np.sum(pd.concat([best_dt_1, best_dt_2, best_dt_3]
                        ).roc_auc_score.values * weighted_average)
e_rf = np.sum(pd.concat([best_rf_1, best_rf_2, best_rf_3]
                        ).roc_auc_score.values * weighted_average)
e_gb = np.sum(pd.concat([best_gb_1, best_gb_2, best_gb_3]
                        ).roc_auc_score.values * weighted_average)

In [64]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
# display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
# display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
# display('knn', pd.concat([best_knn_1,best_knn_2,best_knn_3]))
display('svc', pd.concat([best_svc_1, best_svc_2, best_svc_3]))
# display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
# display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
# display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
# display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

'svc'

f1_score  roc_auc_score  accuracy_score
kernel gamma oversampling                                         
linear 0.10  True          0.385733       0.667996        0.726384
rbf    0.01  True          0.305217       0.562734        0.665426
             False         0.635175       0.657252        0.656571

In [65]:
results = pd.DataFrame(data={'models': ['logistic', 'lda', 'knn', 'svc', 'naive_bayes', 'decision_tree',
                                        'random_forest', 'gradient_boosting'],
                             'auc_best': [best_log.roc_auc_score.values[0], best_lda.roc_auc_score.values[0],
                                          best_knn.roc_auc_score.values[0], best_svc.roc_auc_score.values[0],
                                          best_nb.roc_auc_score.values[0], best_dt.roc_auc_score.values[0],
                                          best_rf.roc_auc_score.values[0], best_gb.roc_auc_score.values[0]],
                             'auc_expert_weighted': [e_log, e_lda, e_knn, e_svc, e_nb, e_dt, e_rf, e_gb]})

display(results)

,models,auc_best,auc_expert_weighted
0,logistic,0.597721,0.621718
1,lda,0.598766,0.623260
2,knn,0.596747,0.623623
3,svc,0.598347,0.626044
4,naive_bayes,0.499562,0.553438
5,decision_tree,0.554414,0.582209
6,random_forest,0.572887,0.612298
7,gradient_boosting,0.580569,0.608280


### Conclusions


In [66]:
# Save dataframe
# results.to_pickle("results_coarse_no_metadata.pkl")

In [67]:
results.mean(axis=0)

auc_best               0.574877
auc_expert_weighted    0.606359
dtype: float64

In [68]:
results.loc[results['auc_best'].argmax()]

models                      lda
auc_best               0.598766
auc_expert_weighted     0.62326
Name: 1, dtype: object

In [69]:
results.loc[results['auc_expert_weighted'].argmax()]

models                      svc
auc_best               0.598347
auc_expert_weighted    0.626044
Name: 3, dtype: object